In [7]:
# Install necessary libraries for reading Excel files
!pip install pandas openpyxl

import pandas as pd

# --- Load the data files into pandas DataFrames ---

try:
    # Main register data
    df_all_time = pd.read_excel('All time data from Register.xlsx')

    # Data on potential non-lodgers
    df_non_lodger = pd.read_excel('ato_tax_transparency_non_lodger.xlsx')

    # Data on entities that have lodged once (assuming this is the primary file)
    # The .csv file might be a simpler version, while the .xlsx has more detail.
    # We will start with the xlsx.
    df_lodge_once = pd.read_excel('lodge_once_cont.xlsx')

    print("All data files loaded successfully!")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure all files are uploaded to the Colab session.")
    print("Files expected: 'All time data from Register.xlsx', 'ato_tax_transparency_non_lodger.xlsx', 'lodge_once_cont.xlsx'")

All data files loaded successfully!


In [4]:
# --- Explore the loaded DataFrames ---

# Check if the DataFrames were loaded before trying to use them
if 'df_all_time' in locals():
    print("--- 1. All time data from Register ---")
    print(f"Shape (rows, columns): {df_all_time.shape}")
    print("First 5 rows:")
    print(df_all_time.head())
    print("\nColumn Info:")
    df_all_time.info()
    print("\n" + "="*50 + "\n")

if 'df_non_lodger' in locals():
    print("--- 2. ATO Tax Transparency Non-Lodger Data ---")
    print(f"Shape (rows, columns): {df_non_lodger.shape}")
    print("First 5 rows:")
    print(df_non_lodger.head())
    print("\nColumn Info:")
    df_non_lodger.info()
    print("\n" + "="*50 + "\n")

if 'df_lodge_once' in locals():
    print("--- 3. Lodge Once Data ---")
    print(f"Shape (rows, columns): {df_lodge_once.shape}")
    print("First 5 rows:")
    print(df_lodge_once.head())
    print("\nColumn Info:")
    df_lodge_once.info()
    print("\n" + "="*50 + "\n")

In [9]:
# --- Load the CSV data file ---
try:
    df_lodge_once_csv = pd.read_csv('lodge_once.csv')
    print("--- Additional File: lodge_once.csv ---")
    print("Load successful.")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure 'lodge_once.csv' is also uploaded.")


# --- Compare lodge_once.csv and lodge_once_cont.xlsx ---
# We'll use the original df_lodge_once DataFrame for a direct comparison before cleaning
if 'df_lodge_once_csv' in locals() and 'df_lodge_once' in locals():
    print("\n--- Comparing the two 'lodge_once' files ---")

    # Compare shape (rows, columns)
    print(f"Shape of lodge_once.csv: {df_lodge_once_csv.shape}")
    print(f"Shape of lodge_once_cont.xlsx: {df_lodge_once.shape}")

    # Compare columns
    csv_columns = set(df_lodge_once_csv.columns)
    xlsx_columns = set(df_lodge_once.columns)

    if csv_columns == xlsx_columns:
        print("\nColumn names are identical in both files.")
    else:
        print("\nColumn names differ.")
        if len(csv_columns - xlsx_columns) > 0:
            print(f"Columns found ONLY in CSV: {csv_columns - xlsx_columns}")
        if len(xlsx_columns - csv_columns) > 0:
            print(f"Columns found ONLY in XLSX: {xlsx_columns - csv_columns}")

    # Quick content check on a key column
    # This helps see if they contain the same data, e.g., the dummy ABNs
    csv_dummy_count = df_lodge_once_csv['abn'].str.startswith('dummy_').sum()
    xlsx_dummy_count = df_lodge_once['abn'].str.startswith('dummy_').sum()

    print(f"\nDummy ABN count in CSV: {csv_dummy_count}")
    print(f"Dummy ABN count in XLSX: {xlsx_dummy_count}")

    if df_lodge_once_csv.shape == df_lodge_once.shape and csv_dummy_count == xlsx_dummy_count:
        print("\nConclusion: The files appear to be identical in structure and content.")
        print("We can proceed using the 'lodge_once_cont.xlsx' file with confidence.")
    else:
        print("\nConclusion: The files have differences that need to be investigated before proceeding.")

--- Additional File: lodge_once.csv ---
Load successful.

--- Comparing the two 'lodge_once' files ---
Shape of lodge_once.csv: (2918, 35)
Shape of lodge_once_cont.xlsx: (2918, 35)

Column names differ.
Columns found ONLY in CSV: {'expected_period_end', 'all_np', 'c_criteria_1f', 'last_period_end', 'num_assessment', 'last_stmt_year', 'num_published', 'repeat_nc', 'nc_criteria_1c', 'c_criteria_1a', 'num_non_publishable', 'all_p', 'alter_nc', 'num_compliant', 'nc_criteria_1b', 'nc_index', 'all_c', 'expected_due_date', 'nc_criteria_1e', 'nc_criteria_1a', 'c_criteria_1e', 'c_criteria_1d', 'nc_criteria_1d', 'all_nc', 'first_nc', 'last_submission_dttm', 'c_criteria_1b', 'c_criteria_1c', 'num_statements', 'num_non_compliant', 'nc_criteria_1f', 'revenue', 'num_publishable'}
Columns found ONLY in XLSX: {'mn_bus_sbrb', 'nm_sufx_cd', 'sprsn_ind', 'pid', 'state', 'mn_bus_pc', 'company_name', 'gst_cancn_dt', 'son_stt', 'prsn_othr_gvn_nm', 'son_sbrb', 'son_dpid', 'son_addr_ln_2', 'prsn_gvn_nm', 'nm_

In [10]:
# --- Step 3 (Revised): Data Cleaning, Merging, and Preparation ---

# It's good practice to work on copies of the data
df_all_time_clean = df_all_time.copy()
df_non_lodger_clean = df_non_lodger.copy()
df_lodge_once_details = df_lodge_once.copy() # From the .xlsx file
df_lodge_once_compliance = df_lodge_once_csv.copy() # From the .csv file

# 1. Clean column names in the main register data
df_all_time_clean.columns = df_all_time_clean.columns.str.strip().str.replace('\n', '')

# 2. Standardize entity names in the register for matching
df_all_time_clean.dropna(subset=['Reporting entities'], inplace=True)
df_all_time_clean['entity_name_clean'] = df_all_time_clean['Reporting entities'].str.upper().str.strip()

# 3. Standardize entity names in the non-lodger list
df_non_lodger_clean['entity_name_clean'] = df_non_lodger_clean['Entity Name'].str.upper().str.strip()

# --- NEW: Clean and Merge the two 'lodge_once' files ---

# 4. Remove 'dummy' records from both dataframes to ensure a clean merge
df_lodge_once_details = df_lodge_once_details[~df_lodge_once_details['abn'].str.startswith('dummy_')].copy()
df_lodge_once_compliance = df_lodge_once_compliance[~df_lodge_once_compliance['abn'].str.startswith('dummy_')].copy()

# 5. Merge the two dataframes on their common key(s). Let's use 'abn'.
# We assume 'abn' is the unique identifier for an entity across these files.
df_lodge_once_merged = pd.merge(
    df_lodge_once_details,
    df_lodge_once_compliance,
    on='abn',
    how='inner' # 'inner' merge ensures we only keep records present in both files
)

# 6. Standardize names in the newly merged single-lodger dataframe
df_lodge_once_merged['entity_name_clean'] = df_lodge_once_merged['company_name'].str.upper().str.strip()


print("\nData cleaning, merging, and preparation complete.")
print(f"Proceeding with {len(df_all_time_clean)} records from the register.")
print(f"Proceeding with {len(df_non_lodger_clean)} potential non-lodger records.")
print(f"Created a merged single-lodger dataset with {len(df_lodge_once_merged)} records and {df_lodge_once_merged.shape[1]} columns.")

# Let's inspect the merged data to be sure
print("\nFirst 5 rows of the merged single-lodger dataframe:")
print(df_lodge_once_merged[['abn', 'company_name', 'num_compliant', 'last_submission_dttm']].head())


Data cleaning, merging, and preparation complete.
Proceeding with 16734 records from the register.
Proceeding with 1346 potential non-lodger records.
Created a merged single-lodger dataset with 2289 records and 70 columns.

First 5 rows of the merged single-lodger dataframe:
           abn                       company_name  num_compliant  \
0  83130964162   NORTHERN STAR (BUNDARRA) PTY LTD            NaN   
1  57635649606   NORTHERN STAR (SINCLAIR) PTY LTD            NaN   
2  91151605417  NORTHERN STAR (SR MINING) PTY LTD            NaN   
3  42601140185   NORTHERN STAR (TALISMAN) PTY LTD            NaN   
4  13004486128                                NaN            NaN   

   last_submission_dttm  
0  2020-09-07T05:33:28Z  
1  2020-09-07T05:33:28Z  
2  2020-09-07T05:33:28Z  
3  2020-09-07T05:33:28Z  
4  2020-09-18T02:28:15Z  


In [ ]:
# --- Step 4: Identify Entities that have never lodged ---

# Create a set of unique entity names from the register for efficient lookup
# This set contains every entity that has lodged at least once.
lodged_entity_names = set(df_all_time_clean['entity_name_clean'])

# Filter the non_lodger dataframe to find entities whose cleaned name is NOT in the set of lodged names
never_lodged_mask = ~df_non_lodger_clean['entity_name_clean'].isin(lodged_entity_names)
df_never_lodged = df_non_lodger_clean[never_lodged_mask].copy()

print(f"Identified {len(df_never_lodged)} entities that have likely never lodged a statement.")

# Let's look at the key details for the first 5 identified non-lodgers
print("\n--- Sample of Identified Non-Lodgers ---")
print(df_never_lodged[['ABN', 'Entity Name', 'Total Income', 'ASX listed?', 'Industry_desc']].head())

In [ ]:
# --- Step 5: Identify and Confirm Single-Lodgement Behaviour ---

# Count the number of statements submitted by each unique entity in the register
lodgement_counts = df_all_time_clean['entity_name_clean'].value_counts()

# Filter this list to find entities that appear exactly once
single_lodger_names_from_register = lodgement_counts[lodgement_counts == 1].index.tolist()

# Create a dataframe of these single lodgers from the main register data
df_single_lodgers_from_register = df_all_time_clean[df_all_time_clean['entity_name_clean'].isin(single_lodger_names_from_register)].copy()

print(f"\nIdentified {len(df_single_lodgers_from_register)} entities that have lodged exactly once, based on the register data.")

# --- Validate and Enrich with our Merged Data ---
# We can now link these entities to the detailed ABN and compliance data we prepared earlier.
# This gives us the rich dataset we need for the deliverable.
df_single_lodgers_enriched = pd.merge(
    df_single_lodgers_from_register,
    df_lodge_once_merged,
    on='entity_name_clean',
    how='left' # Use a 'left' merge to keep all entities from our register list
)

print(f"Enriched the single-lodger list. The new dataframe has {df_single_lodgers_enriched.shape[1]} columns.")

# Display a sample of the final, enriched data
print("\n--- Sample of Enriched Single-Lodger Data ---")
# We select columns from both the register (e.g., 'Reporting Period') and the merged file (e.g., 'abn', 'num_compliant')
print(df_single_lodgers_enriched[[
    'Reporting entities', 'Reporting Period', 'abn', 'company_name', 'last_submission_dttm', 'num_compliant'
]].head())